In [44]:
import csv
import os
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from math import sqrt
import pandas as pd 

In [18]:
path = '/home/deep/Workspace/Customer_Behavior/Dataset.csv'
data = pd.read_csv(path)

In [19]:
data.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [20]:
obj_df = data.select_dtypes(include=['object']).copy()
obj_df.head()

,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Churn
0,7590-VHVEG,Female,Yes,No,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,No
1,5575-GNVDE,Male,No,No,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,No
2,3668-QPYBK,Male,No,No,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,Yes
3,7795-CFOCW,Male,No,No,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),No
4,9237-HQITU,Female,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,Yes


In [21]:
lb_make = LabelEncoder()
obj_df['Gender_Code'] = lb_make.fit_transform(obj_df['gender'])
obj_df['Partner_Code'] = lb_make.fit_transform(obj_df['Partner'])
obj_df['Dependents_Code'] = lb_make.fit_transform(obj_df['Dependents'])
obj_df['PhoneService_Code'] = lb_make.fit_transform(obj_df['PhoneService'])
obj_df['MultipleLines_Code'] = lb_make.fit_transform(obj_df['MultipleLines'])
obj_df['InternetService_Code'] = lb_make.fit_transform(obj_df['InternetService'])
obj_df['OnlineSecurity_Code'] = lb_make.fit_transform(obj_df['OnlineSecurity'])
obj_df['OnlineBackup_Code'] = lb_make.fit_transform(obj_df['OnlineBackup'])
obj_df['DeviceProtection_Code'] = lb_make.fit_transform(obj_df['DeviceProtection'])
obj_df['TechSupport_Code'] = lb_make.fit_transform(obj_df['TechSupport'])
obj_df['StreamingTV_Code'] = lb_make.fit_transform(obj_df['StreamingTV'])
obj_df['StreamingMovies_Code'] = lb_make.fit_transform(obj_df['StreamingMovies'])
obj_df['Contract_Code'] = lb_make.fit_transform(obj_df['Contract'])
obj_df['PaperlessBilling_Code'] = lb_make.fit_transform(obj_df['PaperlessBilling'])
obj_df['PaymentMethod_Code'] = lb_make.fit_transform(obj_df['PaymentMethod'])
obj_df['Churn_Code'] = lb_make.fit_transform(obj_df['Churn'])
obj_df[['Gender_Code','Partner_Code', 'Dependents_Code', 'PhoneService_Code','MultipleLines_Code','InternetService_Code',
        'OnlineSecurity_Code','OnlineBackup_Code', 'DeviceProtection_Code', 'TechSupport_Code', 'StreamingTV_Code', 
       'StreamingMovies_Code', 'Contract_Code', 'PaperlessBilling_Code', 'PaymentMethod_Code', 'Churn_Code']].head(10)

,Gender_Code,Partner_Code,Dependents_Code,PhoneService_Code,MultipleLines_Code,InternetService_Code,OnlineSecurity_Code,OnlineBackup_Code,DeviceProtection_Code,TechSupport_Code,StreamingTV_Code,StreamingMovies_Code,Contract_Code,PaperlessBilling_Code,PaymentMethod_Code,Churn_Code
0,0,1,0,0,1,0,0,2,0,0,0,0,0,1,2,0
1,1,0,0,1,0,0,2,0,2,0,0,0,1,0,3,0
2,1,0,0,1,0,0,2,2,0,0,0,0,0,1,3,1
3,1,0,0,0,1,0,2,0,2,2,0,0,1,0,0,0
4,0,0,0,1,0,1,0,0,0,0,0,0,0,1,2,1
5,0,0,0,1,2,1,0,0,2,0,2,2,0,1,2,1
6,1,0,1,1,2,1,0,2,0,0,2,0,0,1,1,0
7,0,0,0,0,1,0,2,0,0,0,0,0,0,0,3,0
8,0,1,0,1,2,1,0,0,2,2,2,2,0,1,2,1
9,1,0,1,1,0,0,2,2,0,0,0,0,1,0,0,0


In [22]:
data['MonthlyCharges'] = data['MonthlyCharges'].astype(int)
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'])
data['TotalCharges'] = data['TotalCharges'].astype(int)

In [23]:
new_df = pd.DataFrame()
new_df['Gender'] = obj_df['Gender_Code']
new_df['SeniorCitizen'] = data['SeniorCitizen']
new_df['Partner'] = obj_df['Partner_Code']
new_df['Dependents'] = obj_df['Dependents_Code']
new_df['Tenure'] = data['tenure']
new_df['PhoneService'] = obj_df['PhoneService_Code']
new_df['MultipleLines'] = obj_df['MultipleLines_Code']
new_df['InternetService'] = obj_df['InternetService_Code']
new_df['OnlineSecurity'] = obj_df['OnlineSecurity_Code']
new_df['OnlineBackup'] = obj_df['OnlineBackup_Code']
new_df['DeviceProtection'] = obj_df['DeviceProtection_Code']
new_df['TechSupport'] = obj_df['TechSupport_Code']
new_df['StreamingTV'] = obj_df['StreamingTV_Code']
new_df['StreamingMovies'] = obj_df['StreamingMovies_Code']
new_df['Contract'] = obj_df['Contract_Code']
new_df['PaperlessBilling'] = obj_df['PaperlessBilling_Code']
new_df['PaymentMethod'] = obj_df['PaymentMethod_Code']
new_df['MonthlyCharges'] = data['MonthlyCharges']
new_df['TotalCharges'] = data['TotalCharges']

In [24]:
new_df.head()

,Gender,SeniorCitizen,Partner,Dependents,Tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,0,0,1,0,1,0,1,0,0,2,0,0,0,0,0,1,2,29,29
1,1,0,0,0,34,1,0,0,2,0,2,0,0,0,1,0,3,56,1889
2,1,0,0,0,2,1,0,0,2,2,0,0,0,0,0,1,3,53,108
3,1,0,0,0,45,0,1,0,2,0,2,2,0,0,1,0,0,42,1840
4,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,1,2,70,151


In [25]:
target_df = pd.DataFrame()
target_df['Churn'] = obj_df['Churn_Code']

In [26]:
target_df.head()

,Churn
0,0
1,0
2,1
3,0
4,1


In [27]:
X_train, X_test, y_train, y_test = train_test_split(new_df, target_df, test_size=0.25)

In [28]:
X_Train = preprocessing.scale(X_train)
X_Test = preprocessing.scale(X_test)

In [41]:
LR = LogisticRegression(class_weight = 'balanced')
LR.fit(X_Train, y_train)
y_pred = LR.predict(X_Test)

/home/deep/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [42]:
mean_squared_error(y_test, y_pred)

0.26462237365133445

In [43]:
sqrt(mean_squared_error(y_test, y_pred))

0.514414593155496

In [48]:
scores = cross_val_score(LR, new_df, target_df, cv=4, scoring='f1_macro')

/home/deep/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/deep/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/deep/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/deep/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_

In [49]:
scores

array([0.72366564, 0.7343902 , 0.69744307, 0.72207696])